# Assignment 2

**Name - Rahul Singla**<br>
**EID - rs57244**<br>

**Name - Anant Gupta**<br>
**EID - ag78593**<br>

**Name - Ishan Patel**<br>
**EID - ikp97**<br>

**Name - Vinay Pahwa**<br>
**EID - vp7339**<br>

**Name - Suchit Das**<br>
**EID - sd38448**<br>

10:30AM Cohort

# Task B

In [13]:
import pandas as pd
import regex as re
import numpy as np
from scipy import spatial

from sklearn.feature_extraction.text import CountVectorizer 

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy

In [14]:
reviews = pd.read_csv('reviews.csv')
reviews.sample(5)

,product_name,product_review,user_rating
923,Westly,A delicious beer indeed Bright orange color p...,4.56
4929,King Sue Double Dry Hopped,Hazy neon yellow body white cap Smell is hug...,4.23
3971,The Abyss,12 Oz bottle Bottled 4 11 2020 Yeah it s les...,4.93
233,SR 71,This beer takes everything that is lovely abou...,4.88
2528,The Rusty Nail,2020 vintage Pitch black with a faint head I...,4.65


In [15]:
def clean_text(string):
    if pd.isna(string):
        return ""
    string = string.lower()
    string = string.replace("\n", " ").replace("\t", " ")
    string = text_after = re.sub(r'[^\w]', ' ', string)
    return string

In [16]:
reviews['clean_reviews'] = reviews['product_review'].apply(clean_text)

In [17]:
corpus = ""
n = len(reviews)

for i in range(n):
    corpus = corpus + clean_text(reviews["clean_reviews"].iloc[i])

In [18]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform([corpus])
word_count = pd.Series(X.toarray()[0], index=vectorizer.get_feature_names())
ranked_word_count = word_count.sort_values(ascending=False).reset_index()
ranked_word_count.to_csv('word_count.csv')

In [19]:
features = ['hoppy', 'crisp', 'bitter']

# Task C

In [20]:
vectorizer = CountVectorizer(stop_words='english', vocabulary=features)
vectors = vectorizer.transform(reviews['clean_reviews']).toarray()

In [21]:
vectors

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

In [22]:
len(vectors)

6227

In [23]:
hoppy = []
crisp = []
bitter = []

for i in vectors:
    hoppy.append(i[0])
    crisp.append(i[1])
    bitter.append(i[2]) 

In [24]:
review_change = reviews.copy()

In [25]:
review_change['hoppy'] = hoppy
review_change['crisp'] = crisp
review_change['bitter'] = bitter
review_change

,product_name,product_review,user_rating,clean_reviews,hoppy,crisp,bitter
0,Kentucky Brunch Brand Stout,2020 vintage acquired during the pandemic It ...,5.00,2020 vintage acquired during the pandemic it ...,0,0,0
1,Kentucky Brunch Brand Stout,Long time waiting to tick this one and I have...,4.56,long time waiting to tick this one and i have...,0,0,0
2,Kentucky Brunch Brand Stout,This review is for the 2019 batch It was bott...,5.00,this review is for the 2019 batch it was bott...,0,0,0
3,Kentucky Brunch Brand Stout,Supreme maple OD Soooo easy drinking amp we...,5.00,supreme maple od soooo easy drinking amp we...,0,0,0
4,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,5.00,i have now had 4 different years of kbbs and c...,0,0,0
...,...,...,...,...,...,...,...
6222,Dragonsaddle,For me I have experienced all of the hoof hea...,4.75,for me i have experienced all of the hoof hea...,0,0,0
6223,Dragonsaddle,fresh can drank on 11 5 2016 very hazy golden...,4.50,fresh can drank on 11 5 2016 very hazy golden...,0,0,0
6224,Dragonsaddle,1 PINT can Served in an oversized stemless win...,4.40,1 pint can served in an oversized stemless win...,0,0,0
6225,Dragonsaddle,L Pours a dark gold with a nice head that sta...,4.79,l pours a dark gold with a nice head that sta...,0,0,0


In [26]:
review_change.iloc[6219]['clean_reviews']

'poured from the can into a tulip  huge beer  but pretty easy drinking considering the abv  tropical fruit  minimal bitterness  solid mouthfeel  really solid triple ipa '

In [27]:
feature_array = np.array([1, 1, 1])
cosine_scores = []

for array in vectors:
    if array.sum() == 0:
        cosine_scores = cosine_scores + [0]
    else:
        cosine_scores = cosine_scores + [1 - spatial.distance.cosine(array, feature_array)]
print(cosine_scores[-10:])

[0, 0, 0, 0, 0.5773502691896257, 0, 0, 0, 0, 0]


In [28]:
reviews['cosine_score'] = cosine_scores

# Task D

In [29]:
analyzer = SentimentIntensityAnalyzer()

In [30]:
reviews['senti_score'] = reviews['clean_reviews'].apply(lambda s: analyzer.polarity_scores(s)['compound'])

In [31]:
reviews.head()

,product_name,product_review,user_rating,clean_reviews,cosine_score,senti_score
0,Kentucky Brunch Brand Stout,2020 vintage acquired during the pandemic It ...,5.00,2020 vintage acquired during the pandemic it ...,0.0,0.5574
1,Kentucky Brunch Brand Stout,Long time waiting to tick this one and I have...,4.56,long time waiting to tick this one and i have...,0.0,0.1159
2,Kentucky Brunch Brand Stout,This review is for the 2019 batch It was bott...,5.00,this review is for the 2019 batch it was bott...,0.0,0.8316
3,Kentucky Brunch Brand Stout,Supreme maple OD Soooo easy drinking amp we...,5.00,supreme maple od soooo easy drinking amp we...,0.0,0.9153
4,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,5.00,i have now had 4 different years of kbbs and c...,0.0,0.9022


# Task E

In [32]:
beer_rating = pd.pivot_table(data = reviews, index='product_name', 
                             values = ['cosine_score', 'senti_score'],
                             aggfunc='mean')
beer_rating['eval_score'] = beer_rating['senti_score'] + beer_rating['cosine_score']
beer_rating

,cosine_score,senti_score,eval_score
product_name,,,
ucaba,0.046188,0.786188,0.832376
A Deal With The Devil,0.023094,0.775296,0.798390
A Deal With The Devil Double Oak Aged,0.000000,0.842444,0.842444
Aaron,0.046188,0.825540,0.871728
Abner,0.217412,0.835944,1.053356
...,...,...,...
West Ashley,0.023094,0.832844,0.855938
Westly,0.046188,0.758864,0.805052
Wide Awake It s Morning,0.069282,0.635148,0.704430


In [33]:
beer_rating.sort_values(by = ['eval_score'], ascending= False)[:3]

,cosine_score,senti_score,eval_score
product_name,,,
Second Fiddle,0.421296,0.760596,1.181892
Emerald Grouper,0.279380,0.848120,1.127500
Dinner,0.280506,0.834944,1.115450


In [34]:
nlp = spacy.load('en_core_web_lg')

In [35]:
def attribute_similarity(s):
    similarity_score = []
    if len(s) == 0:
        return 0
    for f in features:
        text1 = f
        text2 = s
        doc1 = nlp(text1)
        doc2 = nlp(text2)
        similarity_score = similarity_score + [doc1.similarity(doc2)]
    return np.array(similarity_score).mean()

In [36]:
reviews['word_similarity'] = reviews['clean_reviews'].apply(attribute_similarity)
reviews

,product_name,product_review,user_rating,clean_reviews,cosine_score,senti_score,word_similarity
0,Kentucky Brunch Brand Stout,2020 vintage acquired during the pandemic It ...,5.00,2020 vintage acquired during the pandemic it ...,0.0,0.5574,0.305312
1,Kentucky Brunch Brand Stout,Long time waiting to tick this one and I have...,4.56,long time waiting to tick this one and i have...,0.0,0.1159,0.366800
2,Kentucky Brunch Brand Stout,This review is for the 2019 batch It was bott...,5.00,this review is for the 2019 batch it was bott...,0.0,0.8316,0.352708
3,Kentucky Brunch Brand Stout,Supreme maple OD Soooo easy drinking amp we...,5.00,supreme maple od soooo easy drinking amp we...,0.0,0.9153,0.285829
4,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,5.00,i have now had 4 different years of kbbs and c...,0.0,0.9022,0.324963
...,...,...,...,...,...,...,...
6222,Dragonsaddle,For me I have experienced all of the hoof hea...,4.75,for me i have experienced all of the hoof hea...,0.0,0.9870,0.310978
6223,Dragonsaddle,fresh can drank on 11 5 2016 very hazy golden...,4.50,fresh can drank on 11 5 2016 very hazy golden...,0.0,0.7986,0.406110
6224,Dragonsaddle,1 PINT can Served in an oversized stemless win...,4.40,1 pint can served in an oversized stemless win...,0.0,0.8586,0.439072
6225,Dragonsaddle,L Pours a dark gold with a nice head that sta...,4.79,l pours a dark gold with a nice head that sta...,0.0,0.9806,0.406174


In [37]:
beer_rating = pd.pivot_table(data = reviews, index='product_name', 
                             values = ['cosine_score', 'senti_score', 'word_similarity'],
                             aggfunc='mean')
beer_rating['eval_score_old'] = beer_rating['senti_score'] + beer_rating['cosine_score']
beer_rating['eval_score_new'] = beer_rating['senti_score'] + beer_rating['word_similarity']
beer_rating

,cosine_score,senti_score,word_similarity,eval_score_old,eval_score_new
product_name,,,,,
ucaba,0.046188,0.786188,0.453397,0.832376,1.239585
A Deal With The Devil,0.023094,0.775296,0.451155,0.798390,1.226451
A Deal With The Devil Double Oak Aged,0.000000,0.842444,0.440207,0.842444,1.282651
Aaron,0.046188,0.825540,0.456051,0.871728,1.281591
Abner,0.217412,0.835944,0.443012,1.053356,1.278956
...,...,...,...,...,...
West Ashley,0.023094,0.832844,0.487456,0.855938,1.320300
Westly,0.046188,0.758864,0.442396,0.805052,1.201260
Wide Awake It s Morning,0.069282,0.635148,0.452768,0.704430,1.087916


In [38]:
beer_rating.sort_values(by = ['eval_score_new'], ascending= False)[:3]

,cosine_score,senti_score,word_similarity,eval_score_old,eval_score_new
product_name,,,,,
Cable Car Kriek,0.115470,0.931804,0.442121,1.047274,1.373925
Genealogy Of Morals Bourbon Barrel Aged,0.055754,0.915800,0.445813,0.971554,1.361613
Flora Plum,0.115470,0.892916,0.459384,1.008386,1.352300


# Part F

Recommednation by cosine similarity

In [39]:
beer_rating.sort_values(by = ['eval_score_old'], ascending= False)[:3]

,cosine_score,senti_score,word_similarity,eval_score_old,eval_score_new
product_name,,,,,
Second Fiddle,0.421296,0.760596,0.447488,1.181892,1.208084
Emerald Grouper,0.279380,0.848120,0.479333,1.127500,1.327453
Dinner,0.280506,0.834944,0.473145,1.115450,1.308089


In [40]:
feature_count = review_change.groupby(by='product_name')[['product_review', 'hoppy', 'bitter', 'crisp']].agg(
    {'product_review':'count','hoppy':sum, 'bitter':sum, 'crisp':sum})
feature_count['hoppy'] = feature_count['hoppy'] / 25
feature_count['bitter'] = feature_count['bitter'] / 25
feature_count['crisp'] = feature_count['crisp'] / 25

In [41]:
def testmap(s):
    return s in [' Cable Car Kriek', 
                 ' Genealogy Of Morals   Bourbon Barrel Aged', 
                 ' Flora Plum', 
                 ' Emerald Grouper', 
                 ' Dinner', 
                 ' Pliny The Younger']

In [46]:
feature_count = feature_count.reset_index()

In [49]:
feature_count[feature_count['product_name'].map(testmap)].sort_values('hoppy')

,product_name,product_review,hoppy,bitter,crisp
52,Cable Car Kriek,25,0.00,0.04,0.16
99,Flora Plum,25,0.00,0.00,0.20
114,Genealogy Of Morals Bourbon Barrel Aged,25,0.00,0.08,0.04
74,Dinner,25,0.12,0.36,0.16
94,Emerald Grouper,25,0.16,0.32,0.16
193,Pliny The Younger,25,0.24,0.24,0.28


For analysis, we are compared the top 3 recommended by cosine similarity and word2vec method. So, in total we will compared performance across 6 products.

For these 6 products, we looked at the frequency of attritube/ feature words in the customer reviews.

We can clearly see that products recommended by cosine similarity are out-performing the products recommended using word2vec. So, we can say that for this given problem bag-of-words approach is out-performing word2vec approach.

# Part G

In [43]:
star_rating = pd.pivot_table(data = reviews, index='product_name', 
                             values = ['cosine_score', 'senti_score', 'word_similarity', 'user_rating'],
                             aggfunc='mean')

In [44]:
star_rating.sort_values(by='user_rating', ascending=False)[:3]

,cosine_score,senti_score,user_rating,word_similarity
product_name,,,,
Chemtrailmix,0.000000,0.805236,4.77160,0.414386
Blessed,0.046188,0.757964,4.74880,0.420133
Vanilla Bean Assassin,0.000000,0.856150,4.74625,0.414049


## Difference in Methods

The recommendations change quite a bit between the two methods. The reason is because we are using the entire dataset rather than the products that have the attributes we are looking for ('hoppy', 'crisp','bitter'). Using all of the reviews is not going to recommend anything personal to the user. Just because a lot of people like it doesn't mean that everyone likes it. Just because lots of people like Blonde Beers and that they are highly rated doesn't mean that everyone will like them. This isn't about functionality- it's about taste, which differs person to person. That is why recommending the top rated beers without paying attention to similarity and sentiment isn't a good idea. You can see that the cosine-scores and the word_similarity scores are low even thought the user_ratings are high, further justifying that high rating does not necessarily mean that the user will prefer the product.